# Model 


# import data

In [69]:
import os
import sys
from pyprojroot import here

import pandas as pd

In [70]:
# Read in the data in the previous step
df = pd.read_pickle('..\data\processed\data_clean.pkl')
df.T

,0,1,2,3,4,5,6,7,8,9,...,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133
project_title,density matrix renormalization group simulatio...,adolescent depressive vulnerability through th...,autonomic computing laboratory,laboratory for forest genomics,stoe ipds diffractometer stateoftheart image ...,infrastructure to investigate the role of t ly...,malleable costeffective highperformance comput...,cold regions biogeochemical research,sexual health research facility,role of vagal afferents in nutrient signalling...,...,gene therapy development for musculoskeletal d...,surviving memory in postwar el salvador a coll...,mapping stories global research lab,speciation analyses for elucidating molecular ...,the climatedatadriven design cd3 facility for ...,establishment of the intelligent visualization...,infrastructure for the catalytic synthesis of ...,niagara college nextgen greenhouse excellence,lambton water centre,cambrian college
project_description,proposed infrastructure to consist of a comput...,the proposed infrastructure consisting of onsi...,autonomic computing laboratory will enable res...,the proposed facility will enable the examina...,researchers at mcmaster university intend to u...,allergic and autoimmune diseases such as asthm...,a malleable costeffective highperformance comp...,the proposed research will investigate the bio...,the research infrastructure is required for th...,the objective of the research is to understand...,...,this proposal will identify diseasemodifying g...,surviving memory in postwar el salvador is an ...,his application requests infrastructure to est...,selenium se is an important contaminant releas...,this proposal outlines the equipment necessary...,this application is for funds to develop the i...,the proposed project will develop efficient co...,niagara college looks to expand upon strategic...,through this orf request lambton college will ...,through the battery electric vehicle lab bevl ...
area_primary,763,4,25,63,76,41,76,310,4,49,...,41,87,87,34,23,10,74,NaN,NaN,NaN
discipline_primary,12300.0,15100.0,11800.0,14400.0,13000.0,15200.0,11600.0,13500.0,15100.0,15200.0,...,41800.0,20001.0,22700.0,12900.0,10100.0,11800.0,13100.0,NaN,NaN,NaN
approval_date,2005-10-24 00:00:00,2005-07-22 00:00:00,2005-10-24 00:00:00,2005-07-22 00:00:00,2006-12-21 00:00:00,2006-08-31 00:00:00,2005-07-22 00:00:00,2005-10-24 00:00:00,2005-10-24 00:00:00,2005-10-24 00:00:00,...,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00
lead_research_institution,university of windsor,university of toronto,university of toronto,university of toronto,mcmaster university,mcmaster university,queens university,queens university,queens university,queens university,...,western university,western university,wilfrid laurier university,wilfrid laurier university,york university,york university,york university,niagara college,lambton college,cambrian college
city,windsor,toronto,toronto,toronto,hamilton,hamilton,kingston,kingston,kingston,kingston,...,london,london,waterloo,waterloo,toronto,toronto,toronto,niaraga on the lake,sarnia,sudbury
ontario_commitment,50000,59422,292551,200000,100000,197078,100000,128000,64000,125000,...,387254,52026,57077,200000,140000,114726,160000,892391,597715,1000000
total_project_costs,175708,148557,731377,500000,261369,492695,441005,320000,160000,312500,...,968135,130067,150858,500000,365313,329452,447211,7422302,1494288,2765734
keyword,density matrix renormalization group numerical...,depression adolescence cognitive vulnerability...,scaling availability adaptive selfmanaging sel...,comparative functional genomics molecular biol...,magnetocaloric materials for magnetic refriger...,immunology immunomodulation asthma allergy aut...,reconfigurable architectures selfadaptive comp...,biogeochemistry arctic and alpine environments...,sexual health laboratory p

# Add a prediction column y

In [5]:
df.total_project_costs.min(),df.total_project_costs.max(),df.total_project_costs.mean()

(11852, 58435136, 1473472.2000638163)

In [6]:
df['total_project_costs'] > df.total_project_costs.mean()

0       False
1       False
2       False
3       False
4       False
        ...  
3129    False
3130    False
3131     True
3132     True
3133     True
Name: total_project_costs, Length: 3134, dtype: bool

In [7]:
df["y"] = (df['total_project_costs'] > df.total_project_costs.mean()).astype(int)

# Split the data to train-test

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
x=df.drop(['y'],axis=1)
y=df['y']

In [60]:
x_train,x_test,y_train,y_test =train_test_split(x,y)

In [11]:
len(x_train),len(x_test)

(2350, 784)

In [12]:
x_train.shape,y_train.shape

((2350, 12), (2350,))

In [22]:
x_train['project_title']

2425              muscle metabolism in health and disease
798     extending the research capabilities of the ste...
129     centre for advanced photovoltaic devices and s...
2300    advanced neurotherapeutics involving mri guida...
164              integrative genomics for health research
                              ...                        
2461    advancing microbial cell biology via protein c...
2804          canadian robotics and ai ethical design lab
1579    xray diffractometer for materials characteriza...
489     infrastructure for study of hostpathogen inter...
837     establishment of facilities for the study of m...
Name: project_title, Length: 2350, dtype: object

# Tekens

In [33]:
tokens=[]
for text in x_train['project_title']:
    tokens.append(list(set(text.split())))

In [35]:
len(tokens)

2350

In [36]:
tokens[2345:2346]

[['microbial',
  'biology',
  'cell',
  'advancing',
  'crystallography',
  'via',
  'protein']]

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
vectorizer = TfidfVectorizer(min_df=2)
tfidf = vectorizer.fit_transform(x_train['project_title'])

In [39]:
a=vectorizer.vocabulary_  # converting words to numbers
a.get(2500)

In [44]:
print(tfidf) 

  (0, 439)	0.38928261045633505
  (0, 70)	0.15775452366144507
  (0, 693)	0.3724189023008057
  (0, 764)	0.24749826974804312
  (0, 947)	0.5732471074034612
  (0, 1033)	0.5431473842291836
  (1, 852)	0.15721085680306932
  (1, 659)	0.29905576043473275
  (1, 637)	0.2830417582448469
  (1, 171)	0.39776882962991006
  (1, 250)	0.24874309505019154
  (1, 1471)	0.31482780416775175
  (1, 1099)	0.11549486954704101
  (1, 231)	0.4067638656937219
  (1, 1336)	0.1699958777608271
  (1, 1544)	0.2892475607055636
  (1, 574)	0.43042365930694093
  (1, 70)	0.0995231764945739
  (2, 1522)	0.34464644401016814
  (2, 420)	0.49141635387423105
  (2, 1184)	0.5887745989689169
  (2, 34)	0.34865015775067043
  (2, 620)	0.1581987473415275
  (2, 257)	0.35459469897137863
  (2, 70)	0.14405585960486134
  :	:
  (2346, 405)	0.3854230419221513
  (2346, 70)	0.1295621762639207
  (2347, 430)	0.6277726116439716
  (2347, 1688)	0.5203721202585867
  (2347, 917)	0.37912527203532126
  (2347, 270)	0.4054113223019259
  (2347, 620)	0.16437789504

In [40]:
tfidf.nonzero()[1]

array([ 439,   70,  693, ..., 1099, 1544,   70], dtype=int32)

In [45]:
tfidf[2349,70]

0.09323676946674597

In [46]:
#fit change and fit the model
#transform just return the value of known model
type(tfidf)

scipy.sparse.csr.csr_matrix

In [47]:
tfidf=tfidf.toarray()
type(tfidf)

numpy.ndarray

In [48]:
tfidf.shape

(2350, 1694)

In [49]:
words=vectorizer.get_feature_names()

C:\Users\Atieh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [50]:
len(words)

1694

In [51]:
tfidf.nonzero()[1]

array([  70,  439,  693, ..., 1099, 1491, 1544], dtype=int64)

In [53]:
for col in tfidf.nonzero()[1]:
    if tfidf[0,col] > 0.3:
        print (words[col], '-', tfidf[0,col])

disease - 0.38928261045633505
health - 0.3724189023008057
metabolism - 0.5732471074034612
muscle - 0.5431473842291836
health - 0.3724189023008057
health - 0.3724189023008057
health - 0.3724189023008057
disease - 0.38928261045633505
disease - 0.38928261045633505
muscle - 0.5431473842291836
metabolism - 0.5732471074034612
health - 0.3724189023008057
metabolism - 0.5732471074034612
disease - 0.38928261045633505
disease - 0.38928261045633505
health - 0.3724189023008057
health - 0.3724189023008057
disease - 0.38928261045633505
health - 0.3724189023008057
muscle - 0.5431473842291836
health - 0.3724189023008057
health - 0.3724189023008057
disease - 0.38928261045633505
metabolism - 0.5732471074034612
disease - 0.38928261045633505
disease - 0.38928261045633505
health - 0.3724189023008057
health - 0.3724189023008057
disease - 0.38928261045633505
disease - 0.38928261045633505
health - 0.3724189023008057
disease - 0.38928261045633505
health - 0.3724189023008057
muscle - 0.5431473842291836
metaboli

In [54]:
import numpy as np
np.nonzero(tfidf)


(array([   0,    0,    0, ..., 2349, 2349, 2349], dtype=int64),
 array([  70,  439,  693, ..., 1099, 1491, 1544], dtype=int64))

In [55]:
tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Model

In [56]:
x_train=tfidf

In [57]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [78]:
x_test.shape,x_train.shape

((784, 771), (2350, 12))

In [68]:
y_test.shape,y_train.shape

((784,), (2350,))

In [69]:
vectorizer = TfidfVectorizer(min_df=2)
tfidf_test = vectorizer.fit_transform(x_test['project_title'])

In [76]:
x_test=tfidf_test.toarray()
x_test.shape

(784, 771)

In [74]:
model.score(x_test,y_test)

ValueError: X has 771 features, but LogisticRegression is expecting 1694 features as input.